Cel Projektu:

Zrozumieć, na co użytkownicy wydają pieniądze, które kategorie generują największe koszty
oraz gdzie istnieje potencjał oszczędności.

In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

N = 1200

df = pd.DataFrame({
	"transaction_id": range(1, N+1),
	"user_id": np.random.randint(1, 101, size = N),
	"category": np.random.choice(
		["food", "rent", "transport", "entertainment", "health", "subscriptions"],
		size = N,
		p = [0.3, 0.25, 0.15, 0.15, 0.1, 0.05]
	),
	"amount": np.round(np.random.exponential(scale=80, size = N), 2),
	"payment_method": np.random.choice(
		["card", "cash", "transfer"],
		size = N,
		p=[0.6, 0.25, 0.15]
	),
	"is_recurring": np.random.choice([0, 1], size = N, p=[0.7, 0.3]),
	"month": np.random.choice(
		["Jan", "Feb", "Mar", "Apr", "May", "Jun"],
		size=N
	)
})

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  1200 non-null   int64  
 1   user_id         1200 non-null   int32  
 2   category        1200 non-null   object 
 3   amount          1200 non-null   float64
 4   payment_method  1200 non-null   object 
 5   is_recurring    1200 non-null   int64  
 6   month           1200 non-null   object 
dtypes: float64(1), int32(1), int64(2), object(3)
memory usage: 61.1+ KB


ETAP 1 EDA

	1. Podstawowy opis danych


In [ ]:
df.head()

,transaction_id,user_id,category,amount,payment_method,is_recurring,month
0,1,52,rent,4.62,transfer,1,Mar
1,2,93,rent,153.24,cash,0,Apr
2,3,15,entertainment,20.45,cash,0,Jan
3,4,72,food,37.85,cash,0,Jan
4,5,61,transport,49.55,card,0,Mar


In [ ]:
df.shape

# Nasza tabela danych ma 1200 wierszy i 7 kolumn

(1200, 7)

In [ ]:
df.info()

# Nasze dane są pełne, nie ma braków


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  1200 non-null   int64  
 1   user_id         1200 non-null   int32  
 2   category        1200 non-null   object 
 3   amount          1200 non-null   float64
 4   payment_method  1200 non-null   object 
 5   is_recurring    1200 non-null   int64  
 6   month           1200 non-null   object 
dtypes: float64(1), int32(1), int64(2), object(3)
memory usage: 61.1+ KB


Statystyki kwot (amount)

In [ ]:
df["amount"].describe()

# Średnia wynosi 81.46
# Mediana wynosi 57.92
# Min wynosi 0.02
# Max wynosi 652.71
# kwartyle: 25% to 23.18, 50% to 57.92, 75% to 114.03


count    1200.000000
mean       81.459817
std        82.185594
min         0.020000
25%        23.185000
50%        57.920000
75%       114.032500
max       652.710000
Name: amount, dtype: float64

 3. Rozkład kategorii wydatków

In [14]:
# Liczba transakcji per category

df["category"].value_counts()

category
food             351
rent             299
transport        197
entertainment    187
health           103
subscriptions     63
Name: count, dtype: int64

In [ ]:
# Łączna kwota per category

df.groupby("category")["amount"].sum().sort_values(ascending=False)

# Najwięcej pieniędzy wydajemy na food (jedzenie)
# Najczęściej wydajemy pieniądze na food i na rent


category
food             28487.49
rent             24718.99
transport        15602.96
entertainment    14049.37
health            8845.38
subscriptions     6047.59
Name: amount, dtype: float64

4. Metody płatności

In [21]:
# rozkład payment_method

df["payment_method"].value_counts()

payment_method
card        718
cash        306
transfer    176
Name: count, dtype: int64

In [ ]:
# Średnia kwota transakcji per metoda

df.groupby("payment_method")["amount"].mean().sort_values(ascending=False)

# Wniosek: Najmniej płacą gotówką a najwięcej przelewem


payment_method
transfer    83.193011
card        83.132618
cash        76.537876
Name: amount, dtype: float64

5. Wydatki w czasie (miesiące)


In [36]:
# Liczba transakcji per month

df.groupby("month")["transaction_id"].count().sort_values(ascending=False)


month
Feb    234
Jan    198
May    196
Mar    194
Jun    190
Apr    188
Name: transaction_id, dtype: int64

In [30]:
# Suma wydatków per month

df.groupby("month")["amount"].sum().sort_values(ascending=False)


month
Feb    18370.86
May    18213.64
Jan    16087.43
Apr    15480.04
Mar    14879.11
Jun    14720.70
Name: amount, dtype: float64

6. Recurring vs non-recurring

In [33]:
# Ile % transakcji jest is_recurring = 1

is_recurring = (df["is_recurring"] == 1).mean() * 100
print(f"{is_recurring:.2f} % transakcji jest is_recurring = 1")

29.42 % transakcji jest is_recurring = 1


In [ ]:
# Średnia kwota: recurring vs non_recurring

df.groupby("is_recurring")["amount"].mean()

# Porównanie średniej transakcji:
# - recurring (np. subskrypcje, rachunki)
# - non_recurring (wydatki jednorazowe)
# Pozwala ocenić, które wydatki bardziej obiążają budżet w długim terminie

is_recurring
0    84.239965
1    74.789037
Name: amount, dtype: float64

ETAP 2 - Agregacje per użytkownik & Struktura wydaktów

1. Wydatki per użytkownik

In [ ]:
# Tabela podsumowującą zachowanie finansowe każdego użytkownika

user_summary = df.groupby("user_id").agg(
	total_spend = ("amount", "sum"),
	avg_transaction = ("amount", "mean"),
	max_transaction = ("amount", "max"),
	transactions_count = ("transaction_id", "count")
)

user_summary.head()

# Przechodzimy z poziomu pojedynczych transakcji
# na poziom zachowania użytkowników jako całości.
# Taka tabela jest podstawą dalszej analizy finansowej.

,total_spend,avg_transaction,max_transaction,transactions_count
user_id,,,,
1,1577.82,87.656667,308.90,18
2,1101.76,73.450667,236.69,15
3,1943.17,138.797857,330.02,14
4,1209.74,86.410000,290.32,14
5,1345.78,84.111250,356.65,16


TOP 10 użytkowników według sumy wydatków

In [109]:
# posortowani użytkownicy malejąco po total_spend
# top spenders

user_summary.sort_values("total_spend", ascending = False).head(10)


,total_spend,avg_transaction,max_transaction,transactions_count,spender_segment
user_id,,,,,
75,2167.22,127.483529,369.11,17,high_spender
3,1943.17,138.797857,330.02,14,high_spender
51,1927.37,101.440526,390.81,19,high_spender
62,1912.75,79.697917,652.71,24,high_spender
89,1733.36,108.335000,311.19,16,high_spender
92,1708.17,85.408500,238.25,20,high_spender
27,1683.28,129.483077,602.85,13,high_spender
1,1577.82,87.656667,308.90,18,high_spender
58,1560.27,104.018000,370.10,15,high_spender


3. Recurring vs non-recurring (per user)

In [75]:
# ilu użytkowników ma jakiekolwiek recurring transakcja

recurring_only = df[df["is_recurring"] == 1]

liczba_uzytkownikow = recurring_only["user_id"].nunique()

print(f"Liczba użytkowników z co najmniej jedną cykliczną transakcją: {liczba_uzytkownikow}")

Liczba użytkowników z co najmniej jedną cykliczną transakcją: 97


In [ ]:
# średni wydatek użytkowników z recurring vs bez recurring

df.groupby("is_recurring")["amount"].mean()


is_recurring
0    84.239965
1    74.789037
Name: amount, dtype: float64

4. Koncentracja wydatków

Dla każdego użytkownika obliczamy:
- % wydatków w najdroższej kategorii


In [97]:
# expences_per_category

user_category_spend = df.groupby(["user_id", "category"])["amount"].sum().reset_index()

max_category_spend = user_category_spend.groupby("user_id")["amount"].max()

total_user_spend = df.groupby("user_id")["amount"].sum()

concentration = (max_category_spend / total_user_spend) * 100

print(concentration)


user_id
1      36.543459
2      29.982936
3      33.201933
4      51.294493
5      28.057335
         ...    
96     33.595542
97     36.869844
98     42.202897
99     43.911397
100    41.702247
Name: amount, Length: 100, dtype: float64


5. Klasyfikacja użytkowników

Dodamy do tabeli per użytkownik kolumnę:
- "low_spender" -> total < 500
- "medium_spender" -> 500-1500
- "high_spender" -> > 1500

In [108]:
user_summary["spender_segment"] = np.select(
	[
		user_summary["total_spend"] < 500,
		user_summary["total_spend"].between(500, 1500),
		user_summary["total_spend"] > 1500
	],
	["low_spender", "medium_spender", "high_spender"],
	default="unknown"
)

user_summary

,total_spend,avg_transaction,max_transaction,transactions_count,spender_segment
user_id,,,,,
1,1577.82,87.656667,308.90,18,high_spender
2,1101.76,73.450667,236.69,15,medium_spender
3,1943.17,138.797857,330.02,14,high_spender
4,1209.74,86.410000,290.32,14,medium_spender
5,1345.78,84.111250,356.65,16,medium_spender
...,...,...,...,...,...
96,1085.65,83.511538,282.53,13,medium_spender
97,1030.30,68.686667,193.68,15,medium_spender
98,708.34,88.542500,246.39,8,medium_spender


ETAP 3 - ANALIZA OSZCZĘDNOŚCI

In [116]:
# 1. TOP 10% najdroższych transakcji 


# próg 90 percentyla kwoty amount
prog_90 = df["amount"].quantile(0.90)

# Liczba transakcji

liczba_transakcji = df[df["amount"] > prog_90]
# transakcje powyżej tego progu
suma_top_10 = df[df["amount"] > prog_90]["amount"].sum()

# Suma wszystkich kosztów
suma_calkowita = df["amount"].sum()

# Udział procentowy
procent_udzialu = (suma_top_10 / suma_calkowita) * 100


print(f"90% transakcji jest niższych niż: {prog_90:.2f}")
print(f"Znaleziono {len(liczba_transakcji)} transakcji typu 'High Value'")
print(f"Top 10% najdroższych transakcji stanowi {procent_udzialu:.2f}% całkowitych wydatków.")


90% transakcji jest niższych niż: 182.46
Znaleziono 120 transakcji typu 'High Value'
Top 10% najdroższych transakcji stanowi 33.06% całkowitych wydatków.


In [ ]:
# 2. Kategorie "najdroższe", nie "najczęstsze"

# Mediana kwoty per category
df.groupby("category")["amount"].median().sort_values(ascending=False)

# Najdroższą kategorią jest "subscriptions" z medianą 74.12, potem kategoria "health" z medianą 63.81

category
subscriptions    74.12
health           63.81
food             60.79
rent             54.57
entertainment    54.42
transport        49.55
Name: amount, dtype: float64

In [ ]:
# 3. Recurring 

# % całkowitych wydatków, które są recurring

# Suma absolutnie wszystkich wydatków
suma_total = df["amount"].sum()

# Suma tylko tych, które są recurring (is_recurring == 1)
suma_recurring = df[df["is_recurring"] == 1]["amount"].sum()

# Obliczamy procent
procent_wydatkow = (suma_recurring / suma_total) * 100

# Średnia kwota recurring vs non-recurring
srednie_wydatki = df.groupby("is_recurring")["amount"].mean()

print(f"Wydatki cykliczne stanowią: {procent_wydatkow:.2f}%")
print(f"\nŚrednia transakcja jednorazowa (0): {srednie_wydatki[0]:.2f} PLN")
print(f"Średnia transakcja cykliczna (1):   {srednie_wydatki[1]:.2f} PLN")

Wydatki cykliczne stanowią: 27.01%

Średnia transakcja jednorazowa (0): 84.24 PLN
Średnia transakcja cykliczna (1):   74.79 PLN


In [ ]:
# 4. Użytkownicy o wysokiej koncentracji wydatków
